In [1]:
import numpy
import io
from itertools import permutations

In [11]:
def read_file_data_train(fname, nwordstrain, nwordstest):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
    sentences = []
    tags = []
    
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    idx_line = 0
    is_first_word = 0
    
    for i in range(nwordstrain+nwordstest):
        if i<nwordstrain:
            prev_tag = '<start>'
#             print(content[idx_line])
            while not content[idx_line].startswith('</kalimat'):
                if  not content[idx_line].startswith('<kalimat'):
                    content_part = content[idx_line].split('\t')
                    content_lower = content_part[0].lower()
                    if content_part[1] in tag_count:
                        tag_count[content_part[1]] += 1
                    else:
                        tag_count[content_part[1]] = 1

                    current_word_tag = (content_lower,content_part[1])
                    if current_word_tag in word_tag:
                        word_tag[current_word_tag] += 1
                    else:    
                        word_tag[current_word_tag] = 1

                    if is_first_word == 1:
                        current_tag_trans = ('<start>',content_part[1])
                        is_first_word = 0
                    else:
                        current_tag_trans = (prev_tag,content_part[1])

                    if current_tag_trans in tag_trans:
                        tag_trans[current_tag_trans] += 1
                    else:
                        tag_trans[current_tag_trans] = 1                    
                    prev_tag = content_part[1]   

                else:
                    tag_count['<start>'] += 1
                    is_first_word = 1
                idx_line = idx_line + 1
            idx_line = idx_line+1 
        else:
            sentence=[]
            tag = []
#             print(content[idx_line])
            while not content[idx_line].startswith('</kalimat'):
                if  not content[idx_line].startswith('<kalimat'):
                    content_part = content[idx_line].split('\t')
                    sentence.append(content_lower)
                    tag.append(content_part[1])
                idx_line += 1
            sentences.append(sentence)
            tags.append(tag)
            idx_line += 1
    return tag_count, word_tag, tag_trans, idx_line, sentences, tags

In [12]:
def create_trans_prob_table(tag_trans,tag_count):
    trans_prob={}
    for i in tag_count.keys():
        for j in tag_count.keys():
            if (i,j) in tag_trans:
                trans_prob[(i,j)]=tag_trans[(i,j)]/tag_count[i]
    return trans_prob

In [13]:
def create_emission_prob_table(word_tag,tag_count):
    emission_prob={}
    for word_tag_entry in word_tag.keys():
        current_word = word_tag_entry[0]
        current_tag = word_tag_entry[1]
        emission_prob[(current_word,current_tag)] = word_tag[word_tag_entry]/tag_count[current_tag]
    return emission_prob

In [14]:
tag_count, word_tag, tag_trans, n_line, test_sent, test_tag = read_file_data_train('Indonesian_Manually_Tagged_Corpus_ID.tsv',1000,20)
print("Jumlah Per Tag\n")
print(tag_count)
print("\n")
print("Jumlah Baris\n")
print(n_line)

Jumlah Per Tag

{'<start>': 1000, 'NN': 6245, 'SC': 1124, 'VB': 2758, 'NNP': 3715, 'JJ': 1033, 'RB': 546, 'IN': 2167, 'Z': 2653, 'CD': 2276, 'CC': 776, 'PR': 491, 'PRP': 691, 'MD': 522, 'FW': 397, 'NEG': 123, 'DT': 20, 'NND': 148, 'SYM': 299, 'RP': 18, 'WH': 8, 'OD': 93, 'X': 14, 'UH': 4}


Jumlah Baris

28841


In [15]:
print("Tag Per Kata\n")
print(word_tag)
print("\n")

Tag Per Kata

{('kera', 'NN'): 1, ('untuk', 'SC'): 124, ('amankan', 'VB'): 1, ('pesta olahraga', 'NN'): 1, ('pemerintah', 'NNP'): 8, ('kota', 'NNP'): 3, ('delhi', 'NNP'): 5, ('mengerahkan', 'VB'): 1, ('monyet', 'NN'): 15, ('mengusir', 'VB'): 4, ('monyet-monyet', 'NN'): 2, ('lain', 'JJ'): 29, ('yang', 'SC'): 513, ('berbadan', 'VB'): 2, ('lebih', 'RB'): 82, ('kecil', 'JJ'): 15, ('dari', 'IN'): 244, ('arena', 'NN'): 2, ('pesta olahraga', 'NNP'): 1, ('persemakmuran', 'NNP'): 1, ('.', 'Z'): 981, ('beberapa', 'CD'): 21, ('laporan', 'NN'): 17, ('menyebutkan', 'VB'): 10, ('setidaknya', 'RB'): 2, ('10', 'CD'): 21, ('ditempatkan', 'VB'): 2, ('di', 'IN'): 358, ('luar', 'NN'): 5, ('lomba', 'NN'): 1, ('dan', 'CC'): 462, ('pertandingan', 'NN'): 1, ('ibukota', 'NNP'): 2, ('india', 'NNP'): 6, ('pemkot', 'NNP'): 2, ('memiliki', 'VB'): 19, ('28', 'CD'): 5, ('berencana', 'VB'): 8, ('mendatangkan', 'VB'): 2, ('sejenis', 'NN'): 1, ('negara bagian', 'NNP'): 1, ('rajasthan', 'NNP'): 1, ('jumlah', 'NN'): 20, 

In [ ]:
print("Transisi Tabel\n")
print(tag_trans)
print("\n")

In [ ]:
print("Kalimat Test\n")
print(test_sent[0])
len(test_sent[0])

In [ ]:
print("Tag Test\n")
print(test_tag)

In [ ]:
emission_prob = create_emission_prob_table(word_tag,tag_count)
emission_prob["Kera","NN"]

In [ ]:
a = list(emission_prob.keys())
a[0][0]

In [ ]:
trans_prob_tab = create_trans_prob_table(tag_trans,tag_count)
trans_prob_tab

In [ ]:
def get_the_tag_baseline(sentence):
    tags=[]
    a = list(emission_prob.keys())
    for i in range(len(sentence)):
        for k,v in emission_prob.iteritems():
            if sentence[i]==

In [ ]:
test=[]
temp_v = 0
for k in emission_prob.keys():
    if 'orang'==k[0]:
        test.append(k)
for i in range(len(test)):
    if temp_v<emission_prob[test[i]]:
        temp_v=emission_prob[test[i]]
        temp_k=test[i]

In [ ]:
print(temp_k[1],temp_v)